In [9]:
from pandas import DataFrame, read_sql_query, Int32Dtype
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"

In [7]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    dm.id_patient as id_patient,
    d.case_id,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        now()) AS age,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
'''

sdata = read_sql_query(query,engine,parse_dates=True)
sdata.id_patient = sdata.id_patient.astype(Int32Dtype())

# close the pool of connection
engine.dispose()

In [10]:
dtabarre = DataFrame(
    {
        'case_id':[
            '14172120-aa43-4456-82c3-9c38c5da9aaf',
            'dbb335a7-0a50-4e28-8157-32e8a0f5d2d5',
            '62b7012c-235f-4e0f-90e0-3c29d25fc90e',
            '6ab1b8fb-fec0-4867-8b6d-a1adb80c6b2c',
            '218fc9a2-cc8a-4f91-96a8-45a555754090',
            'a80a7e33-0470-49e9-9fda-871f19750f11',
            '3a91ac53-2ebf-49b6-9103-c6dc03611feb',
            'd9cbb68e-0aa4-4dd1-b017-18f6a57d9946',
            '013684fe-9e5c-436e-84c8-2bd12b374af6',
            '381d07d1-bd47-4430-993e-24eec8af792d',
            'bd4729fc-45b9-4da7-8a88-501d3844acd3',
            '41138249-6f19-4004-8e4a-14d954e32a90',
            'd0455ad9-d926-4637-a332-8477fba206f9',
        ]
    }
)

dtabarre.case_id.count()
sdata.case_id.count()

13

66932

In [12]:
found_case_id = sdata[sdata.case_id.isin(dtabarre.case_id)]
found_case_id

,id_patient,case_id,code,first_name,last_name,age,dob,interview_date,e__telefn,adress,...,interviewer_lastname,interviewer_role,interviewer_other_info,interview_location,is_your_parent_alive,mothers_name,fathers_name,who_is_your_law_parent,total,organisation
324,128298,013684fe-9e5c-436e-84c8-2bd12b374af6,PAP/DRMS/031342801,Shelda,Joseph,16.0,2006-02-16,2021-10-22,31113496,carradeux,...,Louis Jean,mentor,None,village de la Grace,0.0,Sanon Rachel,Joseph Jean Marck,None,32.0,Caris
5223,76977,14172120-aa43-4456-82c3-9c38c5da9aaf,PAP/DRMS/097081306,Kerwine,MEANCE,12.0,2009-10-28,2020-07-14,39947568,carradeux,...,Valestil,mentor,None,Ketia,0.0,Alexandre Meance,Donese,,20.0,CARIS
8625,122958,218fc9a2-cc8a-4f91-96a8-45a555754090,PAP/DRMS/030296503,Guerdie,Merzy,16.0,2005-09-04,2021-08-31,34782560,carradeux,...,Louis Jean,mentor,None,village de la grace,4.0,None,None,grand soeur,19.0,CARIS
14497,130037,381d07d1-bd47-4430-993e-24eec8af792d,PAP/DRMS/215355603,Rachida,Simeon,10.0,2011-07-15,2021-11-26,,tabarre,...,Accy,mentor,None,Tabarre,0.0,,Simeon,,26.0,CARIS
15104,130038,3a91ac53-2ebf-49b6-9103-c6dc03611feb,PAP/DRMS/215355604,Syndia Santa,Surin,14.0,2008-04-05,2021-11-26,,tabarre,...,Accy,mentor,None,tabarre,0.0,,Surin,,26.0,CARIS
16853,84644,41138249-6f19-4004-8e4a-14d954e32a90,PAP/DRMS/031163216,marie Milanne,remy,23.0,1998-05-22,2020-08-15,,,...,Phaëth,mentor,None,,2.0,,Jean felix,,22.0,CARIS
25784,87297,62b7012c-235f-4e0f-90e0-3c29d25fc90e,PAP/DRMS/097161002,Mirlande,Tanis,21.0,2000-11-28,2020-08-15,34262632,Carradeux (Camp Toto),...,Phaëth,mentor,None,Village de la grâce,2.0,Manette,Eldieu,Guerline Mondesir,45.0,CARIS
27908,128301,6ab1b8fb-fec0-4867-8b6d-a1adb80c6b2c,PAP/DRMS/031342804,Fritzlene,Fevrin,17.0,2005-03-03,2021-10-23,48628298,village de la grace,...,Cazeau,mentor,None,carradeux,0.0,Maculene,Frisnel,maman,26.0,Caris
44049,122972,a80a7e33-0470-49e9-9fda-871f19750f11,PAP/DRMS/030296517,Claudiana,Mezy,14.0,2007-10-22,2021-08-06,44245927,carradeux,...,Louis Jean,mentor,None,village de la grace,0.0,Ruthlene,Claude Jean,maman,18.0,CARIS
49580,122977,bd4729fc-45b9-4da7-8a88-501d3844acd3,PAP/DRMS/030296522,Noradine,Normil,11.0,2011-02-28,2021-05-01,36309186,Carradeux,...,Phaeth,mentor,None,village de la grâce,2.0,Françoise Devoline,Noramix Normil,None,25.0,Caris


In [13]:
found_case_id.to_excel('./tabarre_found.xlsx',na_rep='',index=False)